This notebook generates a displacement adjacement matrix for stations.

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from functools import reduce
from geopy import Point
from geopy import distance

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
class AnalysisConfig:
    def __init__(self, csv_output_path='', stations_path=''):
        self.csv_output_path = csv_output_path
        self.stations_path = stations_path

In [3]:
STATIONS_DIR = '../data/exports'
SHAPEFILE_DIR = '../data/shapefiles/zipcodes'
VARIABLE = 'elevation'

config_sf = AnalysisConfig(
    csv_output_path='{}/{}_adjacency_matrix_san_francisco.csv'.format(
        STATIONS_DIR, VARIABLE),
    #stations_path='{}/stations_san_francisco.csv'.format(STATIONS_DIR),
    stations_path='{}/SF_ele_single station.csv'.format(STATIONS_DIR),
)

config_dc = AnalysisConfig(
    csv_output_path='{}/_adjacency_matrix_washington_dc.csv'.format(
        STATIONS_DIR, VARIABLE),
    #stations_path='{}/stations_washington_dc.csv'.format(STATIONS_DIR),
    stations_path='{}/DC_ele_single station.csv'.format(STATIONS_DIR),
)

In [4]:
config = config_dc

In [5]:
def get_distance(point1, point2):
    return distance.geodesic(point1, point2).m


def get_point(station_id):
    return Point(stations_df.loc[station_id]['lat'],
                 stations_df.loc[station_id]['lng'])


def get_displacement(station_id1, station_id2):
    return get_distance(get_point(station_id1), get_point(station_id2))

def get_elevation_change(station_id1, station_id2):
    return (stations_df.loc[station_id2]['DN'] - 
            stations_df.loc[station_id1]['DN'])

In [6]:
stations_df = pd.read_csv(config.stations_path)
stations_df.set_index('station_id', inplace=True)
stations_df.head()

,lat,lng,DN
station_id,,,
31000,38.858970,-77.053239,14.0
31001,38.857250,-77.053402,15.0
31002,38.856408,-77.049262,10.0
31003,38.860856,-77.049425,6.0
31004,38.857866,-77.059490,16.0


In [7]:
adj_matrix = pd.DataFrame(index=stations_df.index, columns=stations_df.index)

if VARIABLE == 'displacement':
    adj_matrix = adj_matrix.apply(lambda row: row.index.to_series().apply(
        lambda col_name: get_displacement(row.name, col_name)),
                                  axis=1)
elif VARIABLE == 'elevation':
    adj_matrix = adj_matrix.apply(lambda row: row.index.to_series().apply(
        lambda col_name: get_elevation_change(row.name, col_name)),
                                  axis=1)
adj_matrix.head()

station_id,31000,31001,31002,31003,31004,31005,31006,31007,31009,31010,...,32417,32418,32419,32420,32421,32422,31711,31715,31815,32409
station_id,,,,,,,,,,,,,,,,,,,,,
31000,0.0,1.0,-4.0,-8.0,2.0,1.0,3.0,0.0,-6.0,-11.0,...,8.0,NaN,14.0,27.0,3.0,19.0,-4.0,26.0,34.0,40.0
31001,-1.0,0.0,-5.0,-9.0,1.0,0.0,2.0,-1.0,-7.0,-12.0,...,7.0,NaN,13.0,26.0,2.0,18.0,-5.0,25.0,33.0,39.0
31002,4.0,5.0,0.0,-4.0,6.0,5.0,7.0,4.0,-2.0,-7.0,...,12.0,NaN,18.0,31.0,7.0,23.0,0.0,30.0,38.0,44.0
31003,8.0,9.0,4.0,0.0,10.0,9.0,11.0,8.0,2.0,-3.0,...,16.0,NaN,22.0,35.0,11.0,27.0,4.0,34.0,42.0,48.0
31004,-2.0,-1.0,-6.0,-10.0,0.0,-1.0,1.0,-2.0,-8.0,-13.0,...,6.0,NaN,12.0,25.0,1.0,17.0,-6.0,24.0,32.0,38.0


In [8]:
adj_matrix.to_csv(config.csv_output_path)

In [9]:
# Example for retrieving displacement from SF.
csv_adj_matrix = pd.read_csv(config.csv_output_path).set_index('station_id')
display(csv_adj_matrix.head())

#csv_adj_matrix['SF-AA15']['SF-Y14']

,31000,31001,31002,31003,31004,31005,31006,31007,31009,31010,...,32417,32418,32419,32420,32421,32422,31711,31715,31815,32409
station_id,,,,,,,,,,,,,,,,,,,,,
31000,0.0,1.0,-4.0,-8.0,2.0,1.0,3.0,0.0,-6.0,-11.0,...,8.0,NaN,14.0,27.0,3.0,19.0,-4.0,26.0,34.0,40.0
31001,-1.0,0.0,-5.0,-9.0,1.0,0.0,2.0,-1.0,-7.0,-12.0,...,7.0,NaN,13.0,26.0,2.0,18.0,-5.0,25.0,33.0,39.0
31002,4.0,5.0,0.0,-4.0,6.0,5.0,7.0,4.0,-2.0,-7.0,...,12.0,NaN,18.0,31.0,7.0,23.0,0.0,30.0,38.0,44.0
31003,8.0,9.0,4.0,0.0,10.0,9.0,11.0,8.0,2.0,-3.0,...,16.0,NaN,22.0,35.0,11.0,27.0,4.0,34.0,42.0,48.0
31004,-2.0,-1.0,-6.0,-10.0,0.0,-1.0,1.0,-2.0,-8.0,-13.0,...,6.0,NaN,12.0,25.0,1.0,17.0,-6.0,24.0,32.0,38.0


KeyError: 'SF-AA15'